In [ ]:
def create_matrix(file):
    fh=open(file,'r')
    import math
    import cmath #We use cmath module to handle complex numbers that arise when AC sources are handled
    s=(fh.read().splitlines()) #The file is opened in read mode and we read the entire data and split it everytime a newline character is detected. We use this instead of readlines to avoid \n
    a=s.index('.circuit')#We store the indices of '.circuit' and '.end' to choose the part of the file that lies in between them and that is required to generate the MNA matrix.
    b=s.index('.end')
    f={}#We create a dictionary to store the AC sources and their frequencies, dictionary comes in handy when multiple AC sources are to be used
    acflag=0#We create a flag to know whether or not AC sources are used in the given circuit
#When the size of the file, in terms of number of lines is greater than the line count of .end, we realise that ac sources are used and create a dictionary to hold all source and frequency pairs
    if b!=len(s)-1:
        if s[b+1][0:3]=='.ac':
            acflag=1
    
    r=v=t=c=l=0
    R=[]
    V=[]
    I=[]
    L=[]
    C=[]
    xx=[]
    for i in (s[a+1:b]):
        xx.append(i.split())
    #From the list that contains the netlist that lies between circuit and end, we select the ones with just resistors, capacitors, inductors, voltage sources, current sources and append them to separate lists that store these values separately
    for i in xx:
        if i[0][0]=='R':
            r+=1
            R.append(i)

        if i[0][0]=='V':
            v+=1
            V.append(i)
        if i[0][0]=='I':
            t+=1
            I.append(i)
        if i[0][0]=='L':
            l+=1
            L.append(i)
        if i[0][0]=='C':
            c+=1
            C.append(i)
#Since the AC data consists of nodes 'n' specified in addition to the node number, we write a separate code that neglects these n strings to give just the number of the given node
    if acflag==1:
        for i in range(b+1, len(s)):
            yy=s[i].split()

            f[yy[1]]=yy[2]
            w=yy[2]
        for i in range(t):
            for j in range(1,3):
                if I[i][j][0]=="n":
                    I[i][j]=I[i][j][1]
        for i in range(r):
            for j in range(1,3):
                if R[i][j][0]=="n":
                    R[i][j]=R[i][j][1]
        for i in range(c):
            for j in range(1,3):
                print(C[i][j])
                if C[i][j][0]=="n":
                    C[i][j]=C[i][j][1]
        for i in range(v):
            for j in range(1,3):
                if V[i][j][0]=="n":
                    V[i][j]=V[i][j][1]
        for i in range(l):
            for j in range(1,3):
                if L[i][j][0]=="n":
                    L[i][j]=L[i][j][1]

#All the matrices may also contain GND instead of 0. In order to facilitate numeric handling, we replace all the grounds with zeroes
    for i in range(r):
        for j in range(1,3):
            if R[i][j]=="GND":
                R[i][j]='0'
    for i in range(v):
        for j in range(1,3):
            if V[i][j]=="GND":
                V[i][j]='0'
    for i in range(t):
        for j in range(1,3):
            if I[i][j]=="GND":
                I[i][j]='0'

    for i in range(c):
        for j in range(1,3):
            if C[i][j]=="GND":
                C[i][j]='0'
    for i in range(l):
        for j in range(1,3):
            if L[i][j]=="GND":
                L[i][j]='0'
#We then create a list x that contains all the nodes, 0 through the last node. This when added with the number of independent voltage sources gives us the size of the vector that we need
    x=[]
    for i in range(r):
        if R[i][1] not in x:
            x.append(R[i][1] )
        if R[i][2] not in x:
            x.append(R[i][2] )
    for i in range(c):
        if C[i][1] not in x:
            x.append(C[i][1] )
        if C[i][2] not in x:
            x.append(C[i][2] )
    for i in range(l):
        if L[i][1] not in x:
            x.append(L[i][1] )
        if L[i][2] not in x:
            x.append(L[i][2] )

    for i in range(v):
        if V[i][1] not in x:
            x.append(V[i][1] )
        if V[i][2] not in x:
            x.append(V[i][2] )
#The list is then sorted to make sure that the numbers are in order, i.e. from 0 to nth node.
    x.sort()
    B=[0]*(len(x)-1)
#Now, we initiate a list B with all zeros, having a length of len(x)-1. This minus 1, indicates that the 0 element in x is ignored. 
    
   
    A=[]
#The elements corresponding to nodes in B are supposed to be 0(Except when there are current sources). The elements corresponding to the currents through the independent voltage sources are supposed to be entered according to the value in the voltage lists.
    for i in V[::-1]:
        B.append(int(i[4]))
    for i in I:
        if int(i[1])!=0:
            B[int(i[1])-1]-=int(i[4])
        if int(i[2])!=0:
            B[int(i[2])-1]+=int(i[4])
#Once list B is completed, with zeros and voltage sources, we now turn to the A matrix

#We iterate through the X list and for all nodes, we carry out the following operations
    for j in range(1,len(x)):
        #First create a list that can be added to the matrix A. In each iteration of the loop, we add a list that has all the necessary values of resistors
        d=[0]*(len(x)-1+v)
        #The length of list d is the size of each row in matrix A
        for i in range(r):
            #We create a loop that iterates over the resistor list and modifies values in required positions in the list d
            if i<r:
                if x[j] in R[i][1:3]:#The resistor list values between 1 and 3 are taken because they hold the nodes between which the resistor lies
                    index=R[i].index(x[j])
                    #The variable 'index' stores the index of the current loop value and gets its index from the resistor list

                    if index==1:
                        index2=2
                    elif index==2:
                        index2=1
                    #The index value of the other element is also updated (It is 2 when the other is 1 and vice versa)
                    d[int(R[i][index])-1]+=float(R[int(i)][3])**-1
                    #The correct position of the list is added with the inverse of the resistor value
                    if int(R[i][index2])==0:#Care is taken to do no additions to the list when 0 is one of the nodal voltage values
                        continue
                    else:
                        #The correct position of the list is added with the inverse of the resistor value of the other node. 
                        d[int(R[i][index2])-1]-=float(R[int(i)][3])**-1
#All of these lists are appended to the martrix A. At this stage, A has all the resistor values updated in it
        A.append(d)
#We next add the current through the voltage sources data in the already existing rows in the matrix
#This is done by changing the values in the extra columns (after len(x)-1). That is, we change the value in v number of columns
    kk=0 #We initate a counter to know the position of the voltage source(within the v number of columns)
    
    
    for j in range(1,len(x)):

        for i in range(v):
                if x[j] in V[i][1:3] and kk<v :
                    kk+=1
#The same logic from resistors is used
                    indexx=V[i].index(x[j])
                    if indexx==1:
                        indexx2=2
                    if indexx==2:
                        indexx2=1
#Care must be taken to make sure that no additions or updations are made when the ground is encountered
                    if int(V[i][indexx])!=0 :
                        if indexx==1:
                            A[int(V[i][indexx])-1][kk+len(x)-2]+=1
                    #When a given node is first in the netlistpositions (2), we assign positive polarity to it
                        if indexx==2:
                            A[int(V[i][indexx])-1][kk+len(x)-2]-=1
                    #When a given node is second in the netlistpositions (2), we assign negative polarity to it
                    if int(V[i][indexx2])!=0 :
                        if indexx2==1:
                             A[int(V[i][indexx2])-1][kk+len(x)-2]+=1
                        if indexx2==2:
                             A[int(V[i][indexx2])-1][kk+len(x)-2]-=1
    #L
#We next update inductor values in the already made A matrix
    for j in range(1,len(x)):
        #Similar logic is applied as in the case of resistors and voltages
        for i in range(l):
                if x[j] in L[i][1:3]:
                    lf=L[i][3]#Variable lf stores the value of the inductor
                    indexx=L[i].index(x[j])
                    if indexx==1:
                        indexx2=2
                    if indexx==2:
                        indexx2=1
                  #  print(indexx,'index')
                  #  print(indexx2,'index2')
                    if int(L[i][indexx])!=0 :
                        if indexx==1:
                            #Instead of adding the inverse of resistor value, we add the inverse of the impedance which is (jwl)
                            #w takes the value of the frequency needed
                            A[j-1][int(L[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))**-1
                        if indexx==2:
                            A[j-1][int(L[i][indexx])-1]-=(1j*2*math.pi*float(w)*float(lf))**-1
                    if int(L[i][indexx2])!=0 :#Similarly, the other node is accounted for as well
                        if indexx2==1:
                             A[j-1][int(L[i][indexx2])-1]+=(1j*2*math.pi*float(w)*float(lf))**-1
                        if indexx2==2:
                             A[j-1][int(L[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))**-1
    #C The same logic is used again in the context of capacitors, but with a different impedance value of 1/(jwc)
    for j in range(1,len(x)):
        for i in range(c):
                if x[j] in C[i][1:3]:

                    lf=C[i][3]
                    print(lf)
                    indexx=C[i].index(x[j])
                    if indexx==1:
                        indexx2=2
                    if indexx==2:
                        indexx2=1
                   # print(indexx,'index')
                   # print(indexx2,'index2')
                    if int(C[i][indexx])!=0 :
                        if indexx==1:

                            A[j-1][int(C[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))
                        if indexx==2:

                            A[j-1][int(C[i][indexx])-1]-=(1j*2*math.pi*float(w)*float(lf))
                    if int(L[i][indexx2])!=0 :
                        if indexx2==1:

                            A[j-1][int(C[i][indexx2])-1]+=(1j*2*math.pi*float(w)*float(lf))
                        if indexx2==2:

                            A[j-1][int(C[i][indexx2])-1]-=(-1j*2*math.pi*float(w)*float(lf))
    for j in range(1,len(x)):
        d=[0]*(len(x)-1+v)
        for i in range(v):
            if x[j] in V[i][1:3]:
                    indexx=V[i].index(x[j])
                    if indexx==1:
                        indexx2=2
                    if indexx==2:
                        indexx2=1
                    if int(V[i][indexx])!=0 :
                        if indexx==1:

                            d[int(V[i][indexx])-1]+=1
                        elif indexx==2:

                            d[int(V[i][indexx])-1]-=1
                    if int(V[i][indexx2])!=0 :

                         if indexx2==1:
                             d[int(V[i][indexx2])-1]+=1
                         elif indexx2==2:
                             d[int(V[i][indexx2])-1]-=1

        if list(set(d))!=[0] and (d not in A):
            A.append(d)

    fh.close()
    print(file)
    print('B=',B)
    print('A=',A)
    print('x=',gauss2_withpivot(A,B))                
listss=['ckt1.netlist','ckt2.netlist','ckt3.netlist','ckt4.netlist','ckt5.netlist','ckt6.netlist','ckt7.netlist']
for i in listss:
    create_matrix(i)

In [2]:
fh=open('ckt6.netlist','r')
import math
import cmath
s=(fh.read().splitlines()) #instead of readlines to avoid \n
a=s.index('.circuit')
b=s.index('.end')
f={}
acflag=0
if b!=len(s)-1:
    if s[b+1][0:3]=='.ac':
        acflag=1
r=v=t=c=l=0
R=[]
V=[]
I=[]
L=[]
C=[]
xx=[]
for i in (s[a+1:b]):
    xx.append(i.split())
for i in xx:
    if i[0][0]=='R':
        r+=1
        R.append(i)
        
    if i[0][0]=='V':
        v+=1
        V.append(i)
    if i[0][0]=='I':
        t+=1
        I.append(i)
    if i[0][0]=='L':
        l+=1
        L.append(i)
    if i[0][0]=='C':
        c+=1
        C.append(i)

if acflag==1:
    for i in range(b+1, len(s)):
        yy=s[i].split()

        f[yy[1]]=yy[2]
        w=yy[2]
    for i in range(t):
        for j in range(1,3):
            if I[i][j][0]=="n":
                I[i][j]=I[i][j][1]
    for i in range(r):
        for j in range(1,3):
            if R[i][j][0]=="n":
                R[i][j]=R[i][j][1]
    for i in range(c):
        for j in range(1,3):
            print(C[i][j])
            if C[i][j][0]=="n":
                C[i][j]=C[i][j][1]
    for i in range(v):
        for j in range(1,3):
            if V[i][j][0]=="n":
                V[i][j]=V[i][j][1]
    for i in range(l):
        for j in range(1,3):
            if L[i][j][0]=="n":
                L[i][j]=L[i][j][1]
                
    
    
print(f)

        
for i in range(r):
    for j in range(1,3):
        if R[i][j]=="GND":
            R[i][j]='0'
for i in range(v):
    for j in range(1,3):
        if V[i][j]=="GND":
            V[i][j]='0'
for i in range(t):
    for j in range(1,3):
        if I[i][j]=="GND":
            I[i][j]='0'

for i in range(c):
    for j in range(1,3):
        if C[i][j]=="GND":
            C[i][j]='0'
for i in range(l):
    for j in range(1,3):
        if L[i][j]=="GND":
            L[i][j]='0'
x=[]
for i in range(r):
    if R[i][1] not in x:
        x.append(R[i][1] )
    if R[i][2] not in x:
        x.append(R[i][2] )
for i in range(c):
    if C[i][1] not in x:
        x.append(C[i][1] )
    if C[i][2] not in x:
        x.append(C[i][2] )
for i in range(l):
    if L[i][1] not in x:
        x.append(L[i][1] )
    if L[i][2] not in x:
        x.append(L[i][2] )
    
for i in range(v):
    if V[i][1] not in x:
        x.append(V[i][1] )
    if V[i][2] not in x:
        x.append(V[i][2] )

x.sort()
B=[0]*(len(x)-1)

#has names of all the nodes
#take one node by one and iterate over resistors list
A=[]

for i in V[::-1]:
    B.append(int(i[4]))
for i in I:
    if int(i[1])!=0:
        B[int(i[1])-1]-=int(i[4])
    if int(i[2])!=0:
        B[int(i[2])-1]+=int(i[4])
print(B)

for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(r):
        
        if i<r:
            if x[j] in R[i][1:3]:
                index=R[i].index(x[j])
              
                if index==1:
                    index2=2
                elif index==2:
                    index2=1
                d[int(R[i][index])-1]+=float(R[int(i)][3])**-1
                if int(R[i][index2])==0:
                    continue
                else:
                    d[int(R[i][index2])-1]-=float(R[int(i)][3])**-1
        
    A.append(d)
kk=0
for j in range(1,len(x)):
    for i in range(v):
            if x[j] in V[i][1:3] and kk<v :
                kk+=1
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        A[int(V[i][indexx])-1][kk+len(x)-2]+=1
                    if indexx==2:
                        A[int(V[i][indexx])-1][kk+len(x)-2]-=1
                if int(V[i][indexx2])!=0 :
                    if indexx2==1:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]+=1
                    if indexx2==2:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]-=1
#L

for j in range(1,len(x)):
    for i in range(l):
            if x[j] in L[i][1:3]:
                lf=L[i][3]
                indexx=L[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('Lj',j,'i',i)
                A[j-1][int(L[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))**-1
                if int(L[i][indexx2])!=0 :
                    A[j-1][int(L[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))**-1
#C
for j in range(1,len(x)):
    for i in range(c):
            if x[j] in C[i][1:3]:
     
                lf=C[i][3]
                indexx=C[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('j',j,'i',i)
            
                A[j-1][int(C[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))
                if int(L[i][indexx2])!=0 :
                    A[j-1][int(C[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))
for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(v):
        if x[j] in V[i][1:3]:
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        
                        d[int(V[i][indexx])-1]+=1
                    elif indexx==2:
                        
                        d[int(V[i][indexx])-1]-=1
                if int(V[i][indexx2])!=0 :
    
                     if indexx2==1:
                         d[int(V[i][indexx2])-1]+=1
                     elif indexx2==2:
                         d[int(V[i][indexx2])-1]-=1

    if list(set(d))!=[0] and (d not in A):
        A.append(d)

fh.close()
print(B)
print(A)

n1
n2
{'V1': '1000'}
[0, 0, 0, 5]
Lj 1 i 0
j 1 i 0
j 2 i 0
[0, 0, 0, 5]
[[6124.03036408769j, 0, 0, 0], [-6283.185307179586j, (0.001+6283.185307179586j), -0.001, 0], [0, -0.001, 0.001, -1], [0, 0, -1, 0]]


In [8]:
fh=open('ckt5.netlist','r')
import math
import cmath
s=(fh.read().splitlines()) #instead of readlines to avoid \n
a=s.index('.circuit')
b=s.index('.end')
f={}
acflag=0
if b!=len(s)-1:
    if s[b+1][0:3]=='.ac':
        acflag=1
r=v=t=c=l=0
R=[]
V=[]
I=[]
L=[]
C=[]
xx=[]
for i in (s[a+1:b]):
    xx.append(i.split())
for i in xx:
    if i[0][0]=='R':
        r+=1
        R.append(i)
        
    if i[0][0]=='V':
        v+=1
        V.append(i)
    if i[0][0]=='I':
        t+=1
        I.append(i)
    if i[0][0]=='L':
        l+=1
        L.append(i)
    if i[0][0]=='C':
        c+=1
        C.append(i)

if acflag==1:
    for i in range(b+1, len(s)):
        yy=s[i].split()

        f[yy[1]]=yy[2]
        w=yy[2]
    for i in range(t):
        for j in range(1,3):
            if I[i][j][0]=="n":
                I[i][j]=I[i][j][1]
    for i in range(r):
        for j in range(1,3):
            if R[i][j][0]=="n":
                R[i][j]=R[i][j][1]
    for i in range(c):
        for j in range(1,3):
            print(C[i][j])
            if C[i][j][0]=="n":
                C[i][j]=C[i][j][1]
    for i in range(v):
        for j in range(1,3):
            if V[i][j][0]=="n":
                V[i][j]=V[i][j][1]
    for i in range(l):
        for j in range(1,3):
            if L[i][j][0]=="n":
                L[i][j]=L[i][j][1]
                
    
    
print(f)

        
for i in range(r):
    for j in range(1,3):
        if R[i][j]=="GND":
            R[i][j]='0'
for i in range(v):
    for j in range(1,3):
        if V[i][j]=="GND":
            V[i][j]='0'
for i in range(t):
    for j in range(1,3):
        if I[i][j]=="GND":
            I[i][j]='0'

for i in range(c):
    for j in range(1,3):
        if C[i][j]=="GND":
            C[i][j]='0'
for i in range(l):
    for j in range(1,3):
        if L[i][j]=="GND":
            L[i][j]='0'
x=[]
for i in range(r):
    if R[i][1] not in x:
        x.append(R[i][1] )
    if R[i][2] not in x:
        x.append(R[i][2] )
for i in range(c):
    if C[i][1] not in x:
        x.append(C[i][1] )
    if C[i][2] not in x:
        x.append(C[i][2] )
for i in range(l):
    if L[i][1] not in x:
        x.append(L[i][1] )
    if L[i][2] not in x:
        x.append(L[i][2] )
    
for i in range(v):
    if V[i][1] not in x:
        x.append(V[i][1] )
    if V[i][2] not in x:
        x.append(V[i][2] )

x.sort()
B=[0]*(len(x)-1)

#has names of all the nodes
#take one node by one and iterate over resistors list
A=[]

for i in V[::-1]:
    B.append(int(i[4]))
for i in I:
    if int(i[1])!=0:
        B[int(i[1])-1]-=int(i[4])
    if int(i[2])!=0:
        B[int(i[2])-1]+=int(i[4])
print(B)

for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(r):
        
        if i<r:
            if x[j] in R[i][1:3]:
                index=R[i].index(x[j])
              
                if index==1:
                    index2=2
                elif index==2:
                    index2=1
                d[int(R[i][index])-1]+=float(R[int(i)][3])**-1
                if int(R[i][index2])==0:
                    continue
                else:
                    d[int(R[i][index2])-1]-=float(R[int(i)][3])**-1
        
    A.append(d)
kk=0
for j in range(1,len(x)):
    for i in range(v):
            if x[j] in V[i][1:3] and kk<v :
                kk+=1
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        A[int(V[i][indexx])-1][kk+len(x)-2]+=1
                    if indexx==2:
                        A[int(V[i][indexx])-1][kk+len(x)-2]-=1
                if int(V[i][indexx2])!=0 :
                    if indexx2==1:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]+=1
                    if indexx2==2:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]-=1
#L

for j in range(1,len(x)):
    for i in range(l):
            if x[j] in L[i][1:3]:
                lf=L[i][3]
                indexx=L[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('Lj',j,'i',i)
                A[j-1][int(L[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))**-1
                if int(L[i][index2])==0:
                    continue
                else:
                    A[j-1][int(L[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))**-1
#C
for j in range(1,len(x)):
    for i in range(c):
            if x[j] in C[i][1:3]:
     
                lf=C[i][3]
                indexx=C[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('j',j,'i',i)
            
                A[j-1][int(C[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))
                if int(C[i][index2])==0:
                    continue
                else:
                    
                    A[j-1][int(C[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))
for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(v):
        if x[j] in V[i][1:3]:
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        
                        d[int(V[i][indexx])-1]+=1
                    elif indexx==2:
                        
                        d[int(V[i][indexx])-1]-=1
                if int(V[i][indexx2])!=0 :
    
                     if indexx2==1:
                         d[int(V[i][indexx2])-1]+=1
                     elif indexx2==2:
                         d[int(V[i][indexx2])-1]-=1

    if list(set(d))!=[0] and (d not in A):
        A.append(d)

fh.close()
print(B)
print(A)

{}
[0, 10]
[0, 10]
[[0.1, -1], [-1, 0]]


In [ ]:
fh=open('ckt6.txt','r')
import math
import cmath
s=(fh.read().splitlines()) #instead of readlines to avoid \n
a=s.index('.circuit')
b=s.index('.end')
f={}
acflag=0
if b!=len(s)-1:
    if s[b+1][0:3]=='.ac':
        acflag=1
r=v=t=c=l=0
R=[]
V=[]
I=[]
L=[]
C=[]
xx=[]
for i in (s[a+1:b]):
    xx.append(i.split())
for i in xx:
    if i[0][0]=='R':
        r+=1
        R.append(i)
        
    if i[0][0]=='V':
        v+=1
        V.append(i)
    if i[0][0]=='I':
        t+=1
        I.append(i)
    if i[0][0]=='L':
        l+=1
        L.append(i)
    if i[0][0]=='C':
        c+=1
        C.append(i)

if acflag==1:
    for i in range(b+1, len(s)):
        yy=s[i].split()

        f[yy[1]]=yy[2]
        w=yy[2]
    for i in range(t):
        for j in range(1,3):
            if I[i][j][0]=="n":
                I[i][j]=I[i][j][1]
    for i in range(r):
        for j in range(1,3):
            if R[i][j][0]=="n":
                R[i][j]=R[i][j][1]
    for i in range(c):
        for j in range(1,3):
            print(C[i][j])
            if C[i][j][0]=="n":
                C[i][j]=C[i][j][1]
    for i in range(v):
        for j in range(1,3):
            if V[i][j][0]=="n":
                V[i][j]=V[i][j][1]
    for i in range(l):
        for j in range(1,3):
            if L[i][j][0]=="n":
                L[i][j]=L[i][j][1]
                
    
    
print(f)

        
for i in range(r):
    for j in range(1,3):
        if R[i][j]=="GND":
            R[i][j]='0'
for i in range(v):
    for j in range(1,3):
        if V[i][j]=="GND":
            V[i][j]='0'
for i in range(t):
    for j in range(1,3):
        if I[i][j]=="GND":
            I[i][j]='0'

for i in range(c):
    for j in range(1,3):
        if C[i][j]=="GND":
            C[i][j]='0'
for i in range(l):
    for j in range(1,3):
        if L[i][j]=="GND":
            L[i][j]='0'


x=[]
for i in range(r):
    if R[i][1] not in x:
        x.append(R[i][1] )
    if R[i][2] not in x:
        x.append(R[i][2] )
for i in range(c):
    if C[i][1] not in x:
        x.append(C[i][1] )
    if C[i][2] not in x:
        x.append(C[i][2] )
for i in range(l):
    if L[i][1] not in x:
        x.append(L[i][1] )
    if L[i][2] not in x:
        x.append(L[i][2] )
    
for i in range(v):
    if V[i][1] not in x:
        x.append(V[i][1] )
    if V[i][2] not in x:
        x.append(V[i][2] )

x.sort()
B=[0]*(len(x)-1)

#has names of all the nodes
#take one node by one and iterate over resistors list
A=[]

for i in V[::-1]:
    B.append(int(i[4]))
for i in I:
    if int(i[1])!=0:
        B[int(i[1])-1]-=int(i[4])
    if int(i[2])!=0:
        B[int(i[2])-1]+=int(i[4])
print(B)

for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(r):
        
        if i<r:
            if x[j] in R[i][1:3]:
                index=R[i].index(x[j])
              
                if index==1:
                    index2=2
                elif index==2:
                    index2=1
                d[int(R[i][index])-1]+=float(R[int(i)][3])**-1
                if int(R[i][index2])==0:
                    continue
                else:
                    d[int(R[i][index2])-1]-=float(R[int(i)][3])**-1
        
    A.append(d)
kk=0
for j in range(1,len(x)):
    for i in range(v):
            if x[j] in V[i][1:3] and kk<v :
                kk+=1
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        A[int(V[i][indexx])-1][kk+len(x)-2]+=1
                    if indexx==2:
                        A[int(V[i][indexx])-1][kk+len(x)-2]-=1
                if int(V[i][indexx2])!=0 :
                    if indexx2==1:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]+=1
                    if indexx2==2:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]-=1
#L

for j in range(1,len(x)):
    for i in range(l):
        if i<l:
            
            if x[j] in L[i][1:3]:
                lf=L[i][3]
                indexx=L[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('L1j',j,'i',i)
                A[j-1][int(L[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))**-1
               
                if int(L[i][indexx2])==0:
                    continue
                else:
                    print('L2j',j,'i',i)
                    A[j-1][int(L[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))**-1
#C
for j in range(1,len(x)):
    for i in range(c):
        if i<c:
            if x[j] in C[i][1:3]:
     
                lf=C[i][3]
                indexx=C[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('j',j,'i',i)
            
                A[j-1][int(C[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))
                if int(C[i][indexx2])==0:
                    continue
                else:
                    
                    A[j-1][int(C[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))
for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(v):
        if x[j] in V[i][1:3]:
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        
                        d[int(V[i][indexx])-1]+=1
                    elif indexx==2:
                        
                        d[int(V[i][indexx])-1]-=1
                if int(V[i][indexx2])!=0 :
    
                     if indexx2==1:
                         d[int(V[i][indexx2])-1]+=1
                     elif indexx2==2:
                         d[int(V[i][indexx2])-1]-=1

    if list(set(d))!=[0] and (d not in A):
        A.append(d)

fh.close()
print(B)
print(A)

In [6]:
fh=open('ckt4.netlist','r')
import math
import cmath
s=(fh.read().splitlines()) #instead of readlines to avoid \n
a=s.index('.circuit')
b=s.index('.end')
f={}
acflag=0
if b!=len(s)-1:
    if s[b+1][0:3]=='.ac':
        acflag=1
r=v=t=c=l=0
R=[]
V=[]
I=[]
L=[]
C=[]
xx=[]
for i in (s[a+1:b]):
    xx.append(i.split())
for i in xx:
    if i[0][0]=='R':
        r+=1
        R.append(i)
        
    if i[0][0]=='V':
        v+=1
        V.append(i)
    if i[0][0]=='I':
        t+=1
        I.append(i)
    if i[0][0]=='L':
        l+=1
        L.append(i)
    if i[0][0]=='C':
        c+=1
        C.append(i)

if acflag==1:
    for i in range(b+1, len(s)):
        yy=s[i].split()

        f[yy[1]]=yy[2]
        w=yy[2]
    for i in range(t):
        for j in range(1,3):
            if I[i][j][0]=="n":
                I[i][j]=I[i][j][1]
    for i in range(r):
        for j in range(1,3):
            if R[i][j][0]=="n":
                R[i][j]=R[i][j][1]
    for i in range(c):
        for j in range(1,3):
            print(C[i][j])
            if C[i][j][0]=="n":
                C[i][j]=C[i][j][1]
    for i in range(v):
        for j in range(1,3):
            if V[i][j][0]=="n":
                V[i][j]=V[i][j][1]
    for i in range(l):
        for j in range(1,3):
            if L[i][j][0]=="n":
                L[i][j]=L[i][j][1]
                
    
    
print(f)

        
for i in range(r):
    for j in range(1,3):
        if R[i][j]=="GND":
            R[i][j]='0'
for i in range(v):
    for j in range(1,3):
        if V[i][j]=="GND":
            V[i][j]='0'
for i in range(t):
    for j in range(1,3):
        if I[i][j]=="GND":
            I[i][j]='0'

for i in range(c):
    for j in range(1,3):
        if C[i][j]=="GND":
            C[i][j]='0'
for i in range(l):
    for j in range(1,3):
        if L[i][j]=="GND":
            L[i][j]='0'


x=[]
for i in range(r):
    if R[i][1] not in x:
        x.append(R[i][1] )
    if R[i][2] not in x:
        x.append(R[i][2] )
for i in range(c):
    if C[i][1] not in x:
        x.append(C[i][1] )
    if C[i][2] not in x:
        x.append(C[i][2] )
for i in range(l):
    if L[i][1] not in x:
        x.append(L[i][1] )
    if L[i][2] not in x:
        x.append(L[i][2] )
    
for i in range(v):
    if V[i][1] not in x:
        x.append(V[i][1] )
    if V[i][2] not in x:
        x.append(V[i][2] )

x.sort()
B=[0]*(len(x)-1)

#has names of all the nodes
#take one node by one and iterate over resistors list
A=[]

for i in V[::-1]:
    B.append(int(i[4]))
for i in I:
    if int(i[1])!=0:
        B[int(i[1])-1]-=int(i[4])
    if int(i[2])!=0:
        B[int(i[2])-1]+=int(i[4])
print(B)

for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(r):
        
        if i<r:
            if x[j] in R[i][1:3]:
                index=R[i].index(x[j])
              
                if index==1:
                    index2=2
                elif index==2:
                    index2=1
                d[int(R[i][index])-1]+=float(R[int(i)][3])**-1
                if int(R[i][index2])==0:
                    continue
                else:
                    d[int(R[i][index2])-1]-=float(R[int(i)][3])**-1
        
    A.append(d)
kk=0
for j in range(1,len(x)):
    for i in range(v):
            if x[j] in V[i][1:3] and kk<v :
                kk+=1
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        A[int(V[i][indexx])-1][kk+len(x)-2]+=1
                    if indexx==2:
                        A[int(V[i][indexx])-1][kk+len(x)-2]-=1
                if int(V[i][indexx2])!=0 :
                    if indexx2==1:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]+=1
                    if indexx2==2:
                         A[int(V[i][indexx2])-1][kk+len(x)-2]-=1
#L

for j in range(1,len(x)):
    for i in range(l):
        if i<l:
            
            if x[j] in L[i][1:3]:
                lf=L[i][3]
                indexx=L[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('L1j',j,'i',i)
                A[j-1][int(L[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))**-1
               
                if int(L[i][indexx2])==0:
                    continue
                else:
                    print('L2j',j,'i',i)
                    A[j-1][int(L[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))**-1
#C
for j in range(1,len(x)):
    for i in range(c):
        if i<c:
            if x[j] in C[i][1:3]:
     
                lf=C[i][3]
                indexx=C[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                print('j',j,'i',i)
            
                A[j-1][int(C[i][indexx])-1]+=(1j*2*math.pi*float(w)*float(lf))
                if int(C[i][indexx2])==0:
                    continue
                else:
                    
                    A[j-1][int(C[i][indexx2])-1]-=(1j*2*math.pi*float(w)*float(lf))
for j in range(1,len(x)):
    d=[0]*(len(x)-1+v)
    for i in range(v):
        if x[j] in V[i][1:3]:
                indexx=V[i].index(x[j])
                if indexx==1:
                    indexx2=2
                if indexx==2:
                    indexx2=1
                if int(V[i][indexx])!=0 :
                    if indexx==1:
                        
                        d[int(V[i][indexx])-1]+=1
                    elif indexx==2:
                        
                        d[int(V[i][indexx])-1]-=1
                if int(V[i][indexx2])!=0 :
    
                     if indexx2==1:
                         d[int(V[i][indexx2])-1]+=1
                     elif indexx2==2:
                         d[int(V[i][indexx2])-1]-=1

    if list(set(d))!=[0] and (d not in A):
        A.append(d)

fh.close()
print(B)
print(A)

{}
[0, 0, 0, 10]
[0, 0, 0, 10]
[[0.5, -0.5, 0, -1], [-0.5, 1.0333333333333332, -0.2, 0], [0, -0.2, 0.30000000000000004, 0], [-1, 0, 0, 0]]
